<a href="https://colab.research.google.com/github/MJRAJ01/Glucose-RL-Controller/blob/master/4_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#to solve this go and read the class lecture on map reduce where we discussed a trivial join example.
#here the join key is playerid and year id

In [12]:
#install mrjob
!pip install mrjob
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Batting.csv to Batting (2).csv
User uploaded file "Batting.csv" with length 6488747 bytes


In [ ]:
from google.colab import files

# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [188]:
#write the magic string to save it as 4_join.py
%%file 4_join.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys


'''
Join the batting and salaries data for Barry Bonds per year.

The output should be the combined CSV string of batting and salaries data (one per year).

Use 'join' as the key for the final output in the reducer.

E.g:
"join"  "bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,41986,PIT,NL,bondsba01,60000"

Schema:
Salaries: yearID	teamID	lgID	playerID	salary
Batting: playerID	yearID	stint	teamID	lgID	G	AB	R	H	2B	3B	HR	RBI	SB	CS	BB	SO

Hints: 
Use split to split the CSV lines (e.g. s = line.split(','))
Both files are sent to the mapper. Use the length of the lines to determine which is which.
'''


class Join(MRJob):
    # Using SORT_VALUES to sort on all fields, not just first. Read about sort values in https://buildmedia.readthedocs.org/media/pdf/mrjob/latest/mrjob.pdf
    SORT_VALUES = True

    def mapper(self, _, line):
        data = line.split(',')
        if len(data) == 5: #salaries data
          symbol = 'S'
          yield (data[0], data[3]) , (symbol, data)
        else: #batting data
          symbol = 'B'
          yield (data[1], data[0]) , (symbol, data)
      
    def reducer(self, key, values):
      values = [x for x in values]
      if len(values) > 1: # our join hit
        source = values[0]
        for value in values[1:]:
          yield key, (source, value)
      else: # our join missed
        pass

    def reducer_comb(self, key, values):
        values = [x for x in values]
        bats = values[0][0][1]
        salary = values[0][1][1]
        if(bats[0] == 'bondsba01'):
          del bats[-1]
          del bats[5]
        final = bats + salary
        comb = ','.join(final)
        yield 'join', comb
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper, reducer=self.reducer),
            MRStep(reducer=self.reducer_comb)
        ]


if __name__ == '__main__':
    Join.run()

Overwriting 4_join.py


## Step 1. Run MR program locally(Colab)

In [189]:
#write the command to run it first to check for error
%%time
!python 4_join.py -r local Batting.csv Salaries.csv > 4_join.out

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/4_join.root.20230402.224524.258352
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/4_join.root.20230402.224524.258352/output
Streaming final output from /tmp/4_join.root.20230402.224524.258352/output...
Removing temp directory /tmp/4_join.root.20230402.224524.258352...
CPU times: user 47.5 ms, sys: 13.2 ms, total: 60.7 ms
Wall time: 7.48 s


In [190]:
!cat 4_join.out

Streaming output truncated to the last 5000 lines.
"join"	"garlajo01,2010,1,SDN,NL,34,34,56,6,12,1,0,0,2,0,0,6,18,0,0,6,1,0,,2010,SDN,NL,garlajo01,4700000"
"join"	"garzama01,2010,1,TBA,AL,33,2,3,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,,2010,TBA,AL,garzama01,3350000"
"join"	"gaudich01,2010,2,NYA,AL,30,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,2010,OAK,AL,gaudich01,700000"
"join"	"gerutjo01,2010,1,MIL,NL,32,32,71,7,14,4,1,2,8,0,1,3,17,0,0,0,0,1,,2010,MIL,NL,gerutjo01,2000000"
"join"	"gervasa01,2010,1,HOU,NL,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,2010,HOU,NL,gervasa01,402500"
"join"	"getzch01,2010,1,KCA,AL,72,72,224,23,53,9,0,0,18,15,2,19,28,1,2,3,0,3,,2010,KCA,AL,getzch01,414500"
"join"	"giambja01,2010,1,COL,NL,87,87,176,17,43,9,0,6,35,2,0,35,47,5,6,0,5,5,,2010,COL,NL,giambja01,1750000"
"join"	"glaustr01,2010,1,ATL,NL,128,128,412,52,99,18,0,16,71,0,0,63,100,2,4,0,4,16,,2010,ATL,NL,glaustr01,1750000"
"join"	"gloadro01,2010,1,PHI,NL,94,94,128,16,36,8,0,6,22,1,0,8,15,3,1,1,0,1,,2010,PHI,NL,gloadro01,1000000"


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Step 2. Run MR program on AWS EMR

### Please copy  Batting.csv and Salaries.csv into your S3 bucket.

In [ ]:
#run and save output as 4_join.out
#note that you can pass two files -- in any order to the mrjob
#you might first check for errors before cascading the output it to 4_join.out
%%time
!python 4_join.py -r emr s3://<s3 url of the batting dataset> s3://<s3 url of the salaries dataset> --cloud-tmp-dir=s3://<s3 url of tmp directory> --cluster-id=<cluster ID> --conf-path <mrjob.conf file path in colab> > 4_join.out

In [ ]:
!cat 4_join.out

In [ ]:
def test_count():
    f = open('4_join.out')
    lines = f.readlines()
    f.close()
    
    found_line = False
    for  line in lines:
        if line[:-1] in [
            '"join"\t"bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,41986,PIT,NL,bondsba01,60000"',
            '"join"\t"bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,4,1986,PIT,NL,bondsba01,60000"']:
            found_line = True
    if not found_line:
        assert False

    
    assert len(lines) == 22

test_count()